In [1]:
# Packages
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
import datetime as dt

In [2]:
# Read in team name to 3 letter code dictionary
with open('../../data/team_name_dictionary.txt', 'r') as f:
    team_name_dict = json.load(f)

In [3]:
# CSV path
path_to_lineups = '../../data/daily/DF_lineups.csv'

# Main DF URL for line combinations
df_url = 'https://www.dailyfaceoff.com/teams/.../line-combinations'

# List of teams that differentiate  each URL
team_list = ['anaheim-ducks', 'arizona-coyotes', 'boston-bruins', 'buffalo-sabres', 'calgary-flames', 'carolina-hurricanes', 'chicago-blackhawks', 'colorado-avalanche', 'columbus-blue-jackets', 'dallas-stars', 'detroit-red-wings', 'edmonton-oilers', 'florida-panthers', 'los-angeles-kings', 'minnesota-wild', 'montreal-canadiens', 'nashville-predators', 'new-jersey-devils', 'new-york-islanders', 'new-york-rangers', 'ottawa-senators', 'philadelphia-flyers', 'pittsburgh-penguins', 'san-jose-sharks', 'seattle-kraken', 'st-louis-blues', 'tampa-bay-lightning', 'toronto-maple-leafs', 'vancouver-canucks', 'vegas-golden-knights', 'washington-capitals', 'winnipeg-jets']

# Basically trick the site to think you are a genuine user?
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [18]:
# If you only want to scrape teams playing today...
today_only = False

if today_only:
    # 3 letter code version
    team_list_3letter = [team_name_dict[team] for team in team_list]

    # Create a dictionary to map codes to team names
    code_to_name = dict(zip(team_list_3letter, team_list))

    # Use hockey reference to find the correct list of teams for today
    #date_of_games = dt.datetime.now().strftime('%Y-%M-%d')
    date_of_games = '2022-10-14'
    schedule_url = 'https://www.hockey-reference.com/leagues/NHL_2023_games.html'

    # Get the schedule from hockey reference for the given season
    schedule = pd.read_html(schedule_url, attrs={'class':'stats_table', 'id':'games'})[0]
    schedule['Visitor'] = schedule['Visitor'].str.lower().replace(team_name_dict)
    schedule['Home'] = schedule['Home'].str.lower().replace(team_name_dict)

    # Filter for date == date of interest
    schedule = schedule[schedule['Date'] == date_of_games]
    teams_on_date_3letter = schedule['Visitor'].tolist()
    teams_on_date_3letter.extend(schedule['Home'].tolist())

    # Get team names for the interested codes
    team_list = [code_to_name[code] for code in teams_on_date_3letter]

    display(team_list)

In [8]:
# Function to process forwards section
def process_forwards(section_html):
    # Names
    names_html = section_html.find_all(class_ = 'text-xs font-bold uppercase xl:text-base')
    names = [n.text.strip() for n in names_html]

    assert len(names) == 12, f'Incorrect number of forwards were found: {len(names)}.'

    # Positions
    positions = ['LW', 'C', 'RW'] * 4

    # Line number
    line_num = [item for item in [1, 2, 3, 4] for _ in range(3)]

    # Injury status
    injury_html = section_html.find_all(class_ = 'rounded-md bg-red-500 pl-1 pr-1 text-2xl uppercase text-white')
    injury_status = [inj.text.strip().upper() for inj in injury_html]
    injury_names = [inj.find_next('div').find_next('span').text.strip() for inj in injury_html]

    # Game time decision image (doesn't have text/letters)
    gtd_html = section_html.find_all('svg')
    gtd_names = [gtd.find_next('div').find_next('span').text.strip() for gtd in gtd_html]

    # Update the injury lists with GTD info
    if len(gtd_names) > 0:
        injury_names.extend(gtd_names)
        injury_status.extend(['GTD'] * len(gtd_names))

    assert len(injury_status) == len(injury_names), f'Lengths of injury names ({len(injury_names)}) and injury status ({len(injury_status)}) do not match.'
    assert all(name in names for name in injury_names), 'Not all injury names are found in names masterlist.'

    # Create dictionary of information to return
    forward_dict = {
        'names':names,
        'positions':positions,
        'line_num':line_num,
        'injuries':{'name':injury_names, 'injury_status':injury_status}
    }

    return forward_dict

In [7]:
# Function to process defenseman
def process_defenseman(section_html):
    # Names
    names_html = section_html.find_all(class_ = 'text-xs font-bold uppercase xl:text-base')
    names = [n.text.strip() for n in names_html]

    assert len(names) == 6, f'Incorrect number of defenseman were found: {len(names)}.'

    # Positions
    positions = ['LD', 'RD'] * 3

    # Line number
    line_num = [item for item in [1, 2, 3] for _ in range(2)]

    # Injury status
    injury_html = section_html.find_all(class_ = 'rounded-md bg-red-500 pl-1 pr-1 text-2xl uppercase text-white')
    injury_status = [inj.text.strip().upper() for inj in injury_html]
    injury_names = [inj.find_next('div').find_next('span').text.strip() for inj in injury_html]

    # Game time decision image (doesn't have text/letters)
    gtd_html = section_html.find_all('svg')
    gtd_names = [gtd.find_next('div').find_next('span').text.strip() for gtd in gtd_html]

    # Update the injury lists with GTD info
    if len(gtd_names) > 0:
        injury_names.extend(gtd_names)
        injury_status.extend(['GTD'] * len(gtd_names))

    assert len(injury_status) == len(injury_names), f'Lengths of injury names ({len(injury_names)}) and injury status ({len(injury_status)}) do not match.'
    assert all(name in names for name in injury_names), 'Not all injury names are found in names masterlist.'

    # Create dictionary of information to return
    defense_dict = {
        'names':names,
        'positions':positions,
        'line_num':line_num,
        'injuries':{'name':injury_names, 'injury_status':injury_status}
    }

    return defense_dict

In [8]:
# Function to process power play sections
def process_power_play(section_html, unit_num):
    # Names
    names_html = section_html.find_all(class_ = 'text-xs font-bold uppercase xl:text-base')
    names = [n.text.strip() for n in names_html]

    assert len(names) == 5, f'Incorrect number of power play skaters were found: {len(names)}.'

    # Unit number
    unit = [unit_num] * 5

    # Unit position
    pp_position = [1, 2, 3, 4, 5]

    # Create dictionary
    pp_dict = {
        'name':names,
        'pp_unit_num':unit,
        'pp_position':pp_position
    }

    return pp_dict

In [9]:
# Function to process penalty kill sections
def process_penalty_kill(section_html, unit_num):
    # Names
    names_html = section_html.find_all(class_ = 'text-xs font-bold uppercase xl:text-base')
    names = [n.text.strip() for n in names_html]

    assert len(names) == 4, f'Incorrect number of penalty killers were found: {len(names)}.'

    # Unit number
    unit = [unit_num] * 4

    # Unit position
    pk_position = [1, 2, 3, 4]

    # Create dictionary
    pk_dict = {
        'name':names,
        'pk_unit_num':unit,
        'pk_position':pk_position
    }

    return pk_dict

In [10]:
# Function to process goalie section
def process_goalies(section_html):
    # Names
    names_html = section_html.find_all(class_ = 'text-xs font-bold uppercase xl:text-base')
    names = [n.text.strip() for n in names_html]

    assert len(names) in (1, 2, 3), f'Too many goalies were found: {len(names)}.'

    # Positions
    positions = ['G'] * len(names)

    # Depth number
    depth_num = [i + 1 for i in range(len(names))]

    # Injury status
    injury_html = section_html.find_all(class_ = 'rounded-md bg-red-500 pl-1 pr-1 text-2xl uppercase text-white')
    injury_status = [inj.text.strip().upper() for inj in injury_html]
    injury_names = [inj.find_next('div').find_next('span').text.strip() for inj in injury_html]

    # Game time decision image (doesn't have text/letters)
    gtd_html = section_html.find_all('svg')
    gtd_names = [gtd.find_next('div').find_next('span').text.strip() for gtd in gtd_html]

    # Update the injury lists with GTD info
    if len(gtd_names) > 0:
        injury_names.extend(gtd_names)
        injury_status.extend(['GTD'] * len(gtd_names))

    assert len(injury_status) == len(injury_names), f'Lengths of injury names ({len(injury_names)}) and injury status ({len(injury_status)}) do not match.'
    assert all(name in names for name in injury_names), 'Not all injury names are found in names masterlist.'

    # Create dictionary of information to return
    goalie_dict = {
        'names':names,
        'positions':positions,
        'depth_num':depth_num,
        'injuries':{'name':injury_names, 'injury_status':injury_status}
    }
    
    return goalie_dict

In [11]:
# Function to process injury section
def process_injuries(section_html):
    # Injury status
    injury_html = section_html.find_all(class_ = 'rounded-md bg-red-500 pl-1 pr-1 text-2xl uppercase text-white')
    injury_status = [inj.text.strip().upper() for inj in injury_html]
    injury_names = [inj.find_next('div').find_next('span').text.strip() for inj in injury_html]

    ######
    # Not sure if GTD info is really required for this section. Anyone who is located here is probably not GTD?
    # Game time decision image (doesn't have text/letters)
    gtd_html = section_html.find_all('svg')
    gtd_names = [gtd.find_next('div').find_next('span').text.strip() for gtd in gtd_html]

    # Update the injury lists with GTD info
    if len(gtd_names) > 0:
        injury_names.extend(gtd_names)
        injury_status.extend(['GTD'] * len(gtd_names))
    #####
    
    assert len(injury_status) == len(injury_names), f'Lengths of injury names ({len(injury_names)}) and injury status ({len(injury_status)}) do not match.'

    injury_dict = {
        'name':injury_names,
        'injury_status':injury_status
    }
    
    return injury_dict

In [12]:
# For each team, grab the lineup
def get_team_lineup(team):
    # Record the HTML code from url as bs4 object
    response = requests.get(df_url.replace('...', team), headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Get each section individually (forwards, defense, pp1, pp2, etc.)
    sections = soup.find_all(class_ = 'text-3xl text-white')

    # Use functions for each section of webpage
    for sec in sections:
        sec_name = sec.text.strip().lower()
        parent_html = sec.find_parent('div').find_parent('div')

        if sec_name == 'forwards':
            forwards = process_forwards(parent_html)
        elif sec_name == 'defensive pairings':
            defenseman = process_defenseman(parent_html)
        elif sec_name == '1st powerplay unit':
            pp1 = process_power_play(parent_html, unit_num = 1)
        elif sec_name == '2nd powerplay unit':
            pp2 = process_power_play(parent_html, unit_num = 2)
        elif sec_name == '1st penalty kill unit':
            pk1 = process_penalty_kill(parent_html, unit_num = 1)
        elif sec_name == '2nd penalty kill unit':
            pk2 = process_penalty_kill(parent_html, unit_num = 2)
        elif sec_name == 'goalies':
            goalies = process_goalies(parent_html)
        elif sec_name == 'injuries':
            injuries = process_injuries(parent_html)
    
    # Create lineup for each team
    lineup = pd.DataFrame({
        'player_id':np.nan,
        'name':forwards['names'] + defenseman['names'] + goalies['names'],
        'team':team,
        'date_recorded':date_recorded,
        'time_recorded':time_recorded,
        'position':forwards['positions'] + defenseman['positions'] + goalies['positions'],
        'depth_chart_rank':forwards['line_num'] + defenseman['line_num'] + goalies['depth_num']
    })

    # Set up injury data frame to merge
    injuries = pd.concat([pd.DataFrame(forwards['injuries']), pd.DataFrame(defenseman['injuries']), pd.DataFrame(goalies['injuries'])])

    # Merge in injury information
    lineup = pd.merge(lineup, injuries, how = 'left', on = 'name')

    # Set up pp data frame
    power_play = pd.concat([pd.DataFrame(pp1), pd.DataFrame(pp2)])
    penalty_kill = pd.concat([pd.DataFrame(pk1), pd.DataFrame(pk2)])

    # Special teams merge
    lineup = pd.merge(lineup, power_play, how = 'left', on = 'name')
    lineup = pd.merge(lineup, penalty_kill, how = 'left', on = 'name')

    # Cleanup
    lineup[['pp_unit_num', 'pp_position', 'pk_unit_num', 'pk_position']] = lineup[['pp_unit_num', 'pp_position', 'pk_unit_num', 'pk_position']].astype('Int64')
    lineup['team'] = lineup['team'].str.lower().replace(team_name_dict)
    
    return lineup

In [19]:
# Date and time of recording
dt_now = dt.datetime.now()
date_recorded = dt_now.date()
time_recorded = dt_now.time().strftime(format = '%H:%M:%S')

# Get the lineup for each team and store df's in list
lineups_list = list(map(get_team_lineup, team_list))

# Conactenate into a final data frame
lineups_all_teams = pd.concat(lineups_list, ignore_index = True)

In [20]:
display(lineups_all_teams)

,player_id,name,team,date_recorded,time_recorded,position,depth_chart_rank,injury_status,pp_unit_num,pp_position,pk_unit_num,pk_position
0,NaN,Adam Henrique,ANA,2023-09-17,12:17:00,LW,1,NaN,1,1,2,1
1,NaN,Trevor Zegras,ANA,2023-09-17,12:17:00,C,1,NaN,1,5,<NA>,<NA>
2,NaN,Troy Terry,ANA,2023-09-17,12:17:00,RW,1,NaN,1,3,<NA>,<NA>
3,NaN,Alex Killorn,ANA,2023-09-17,12:17:00,LW,2,NaN,2,1,<NA>,<NA>
4,NaN,Mason McTavish,ANA,2023-09-17,12:17:00,C,2,NaN,1,2,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...
635,NaN,Neal Pionk,WPG,2023-09-17,12:17:00,RD,2,NaN,2,4,2,4
636,NaN,Dylan Samberg,WPG,2023-09-17,12:17:00,LD,3,NaN,<NA>,<NA>,2,3
637,NaN,Nate Schmidt,WPG,2023-09-17,12:17:00,RD,3,NaN,2,5,<NA>,<NA>
638,NaN,Connor Hellebuyck,WPG,2023-09-17,12:17:00,G,1,NaN,<NA>,<NA>,<NA>,<NA>


In [44]:
# List of checks we could implement
assert lineups_all_teams.groupby("team")['name'].nunique().unique() == np.array([20]), 'hey'

lineups_all_teams[lineups_all_teams['name'].isna()]

pp_unit_check = lineups_all_teams.groupby('team')['pp_unit_num'].nunique()
x = list(pp_unit_check[pp_unit_check != 2].index)

pp_name_check = lineups_all_teams[~lineups_all_teams['pp_unit_num'].isna()].groupby('team')['name'].nunique()
y = list(pp_name_check[pp_name_check != 10].index)

In [35]:
# Save to CSV
try:
    old_lineups = pd.read_csv(path_to_lineups)
    new_lineups = pd.concat([old_lineups, lineups_all_teams], axis=0).reset_index(drop=True)
    new_lineups.to_csv(path_to_lineups, header=True, index=False)
except:
    raise Exception('Whoops.')

### Where I've left off:
* Don't know how or if I will include the injury section in the data frame.
* Dont know how I will deal with people that have 2 injury codes (ex: DTD + GTD)? Does this ever occur?
* What is the correct way to map player name to player ID?

In [76]:
# Record the HTML code from url as bs4 object
response = requests.get(df_url.replace('...', 'edmonton-oilers'), headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Get each section individually (forwards, defense, pp1, pp2, etc.)
sections = soup.find_all(class_ = 'text-3xl text-white')

In [77]:
forward_section = sections[0]
parent_html = forward_section.find_parent('div').find_parent('div')

In [78]:
lines = parent_html.find_all(class_ = 'mb-4 flex flex-row flex-wrap justify-evenly border-b')
lines.append(parent_html.find_next(class_ = 'flex flex-row flex-wrap justify-evenly').find_next(class_ = 'flex flex-row flex-wrap justify-evenly'))

In [79]:
lines

[<div class="mb-4 flex flex-row flex-wrap justify-evenly border-b"><div class="w-1/3 text-center xl:w-48"><div class="relative flex flex-col items-center"><div class="relative h-20 w-20 xl:h-32 xl:w-32"><a href="/players/news/evander-kane/65"><img alt="Evander Kane" data-nimg="fill" decoding="async" loading="lazy" sizes=" (max-width: 768px) 50vw, (max-width: 1200p) 10vw, 50vw" src="/_next/image?url=https%3A%2F%2Fapi.dailyfaceoff.com%2Fuploads%2Fplayer%2Fjersey%2F65%2FKane-Evander_EDM-2022_.png&amp;w=3840&amp;q=75" srcset="/_next/image?url=https%3A%2F%2Fapi.dailyfaceoff.com%2Fuploads%2Fplayer%2Fjersey%2F65%2FKane-Evander_EDM-2022_.png&amp;w=64&amp;q=75 64w, /_next/image?url=https%3A%2F%2Fapi.dailyfaceoff.com%2Fuploads%2Fplayer%2Fjersey%2F65%2FKane-Evander_EDM-2022_.png&amp;w=96&amp;q=75 96w, /_next/image?url=https%3A%2F%2Fapi.dailyfaceoff.com%2Fuploads%2Fplayer%2Fjersey%2F65%2FKane-Evander_EDM-2022_.png&amp;w=128&amp;q=75 128w, /_next/image?url=https%3A%2F%2Fapi.dailyfaceoff.com%2Fuploa

In [80]:
standard_positions = ['LW', 'C', 'RW']
names = []
positions = []
line_num = []

for line_num_zero_index, line in enumerate(lines):
    containers = line.find_all(class_ = 'w-1/3 text-center xl:w-48')
    for ind, container in enumerate(containers):
        name_html = container.find(class_ = 'text-xs font-bold uppercase xl:text-base')
        if name_html is not None:
            name = name_html.text.strip()
            names.append(name)
            positions.append(standard_positions[ind])
            line_num.append(line_num_zero_index + 1)
        else:
            continue


In [81]:
print(names)
print(positions)
print(line_num)

['Evander Kane', 'Connor McDavid', 'Connor Brown', 'Ryan Nugent-Hopkins', 'Leon Draisaitl', 'Zach Hyman', 'Dylan Holloway', 'Ryan McLeod', 'Warren Foegele', 'Mattias Janmark', 'Derek Ryan']
['LW', 'C', 'RW', 'LW', 'C', 'RW', 'LW', 'C', 'RW', 'LW', 'C']
[1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4]


In [82]:
injury_html = parent_html.find_all(class_ = 'rounded-md bg-red-500 pl-1 pr-1 text-2xl uppercase text-white')
injury_status = [inj.text.strip().upper() for inj in injury_html]
injury_names = [inj.find_next('div').find_next('span').text.strip() for inj in injury_html]

In [83]:
print(injury_status)
injury_names

[]


[]

In [84]:
# Game time decision image (doesn't have text/letters)
gtd_html = parent_html.find_all('svg')
gtd_names = [gtd.find_next('div').find_next('span').text.strip() for gtd in gtd_html]

In [85]:
print(gtd_names)

['Ryan McLeod']
